<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/fine-tuning-experiments/askpatient/askApatient_4_tuning_cadec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
base_dir = 'medical-concept-normalization'

In [27]:
os.mkdir(base_dir)

FileExistsError: ignored

In [28]:
os.mkdir('medical-concept-normalization/data_collection')

FileExistsError: ignored

**Install fastai and clone the fastai for text classification**



In [29]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   287  100   287    0     0   1688      0 --:--:-- --:--:-- --:--:--  1688
bash: line 1: syntax error near unexpected token `newline'
bash: line 1: `<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">'


In [30]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

fatal: destination path 'demo-nlp-classification-fastai' already exists and is not an empty directory.


In [0]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [0]:
path = Path('medical-concept-normalization/data_collection')

In [0]:
# training data set

train_csv = path/'AskAPatient.fold-4.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [34]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-4.validation.csv",header=None)
valid.head()

,0,1
0,6,diastolic dysfunction
1,8,ARTHRITIS
2,8,arthritis
3,8,arthritis
4,8,arthritis


In [35]:
# testing data set

test = pd.read_csv(path/"AskAPatient.fold-4.test.csv",header=None)
test.head()

,0,1
0,4,purple sploches around waist
1,6,stage 1 diastolic dysfunction
2,8,arthritis
3,12,shingles
4,16,flu


In [0]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'fine_tuning_language_model_cadec_data.csv')

In [0]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas.add_test(test,label=0)


In [0]:
data_lm.save()
data_clas.save()

In [0]:
data_lm.train_ds.vocab.itos

In [0]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [0]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [44]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.864302,4.154896,0.223980,03:35


In [0]:
# learn.fit_one_cycle(1, 1e-2)

In [0]:
# learn.fit_one_cycle(1, 1e-3)

In [0]:
# learn.fit_one_cycle(1, 1e-3)

In [0]:
learn.save_encoder('ft_fold4_first')

In [49]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('ft_fold4_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.065545,3.485601,0.407005,01:08


In [50]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.015957,2.873881,0.512077,01:19


In [51]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.513572,2.466453,0.572464,03:24


In [52]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.177238,2.179327,0.628019,03:29
1,1.877086,2.050969,0.654589,03:20


In [53]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.715676,1.881665,0.672705,03:29
1,1.477790,1.806630,0.684783,03:12


In [54]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.378417,1.685531,0.706522,03:30
1,1.241903,1.605139,0.723430,03:20


In [55]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.212217,1.560960,0.736715,03:19
1,1.128758,1.495888,0.745169,03:09


In [56]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.073333,1.490928,0.741546,03:04
1,0.957333,1.413967,0.762077,03:10


In [57]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.900038,1.382743,0.771739,03:30
1,0.944228,1.422630,0.759662,03:09
2,0.863147,1.332305,0.782609,03:16
3,0.742168,1.313333,0.786232,03:27


In [58]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.772409,1.316038,0.782609,03:28
1,0.784487,1.344186,0.783816,03:24
2,0.691819,1.311039,0.795894,03:20
3,0.646746,1.299504,0.794686,03:09


In [59]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.631224,1.305541,0.794686,03:10
1,0.696983,1.341912,0.793478,03:08
2,0.637591,1.299685,0.800725,03:16
3,0.575025,1.296826,0.804348,03:27


In [60]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.585066,1.313485,0.801932,03:16
1,0.636357,1.305048,0.805556,03:12
2,0.548481,1.279826,0.805556,03:08
3,0.537491,1.302933,0.801932,03:09


In [0]:
# save the best model

learn.save_encoder('ask-4-tuning-cadec')

# Part three: Predict on the test dataset

In [0]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [63]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

866
866
[824, 753, 269, 12, 16, 359, 298, 298, 27, 29, 31, 31, 31, 31, 31, 32, 775, 491, 37, 37, 37, 289, 37, 42, 1002, 780, 49, 49, 100, 49, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 53, 928, 53, 928, 109, 471, 61, 61, 62, 710, 76, 76, 76, 614, 289, 614, 753, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 614, 289, 80, 80, 788, 268, 80, 753, 92, 92, 92, 94, 94, 94, 453, 453, 453, 453, 453, 453, 453, 453, 453, 453, 75, 94, 94, 373, 710, 96, 100, 100, 100, 295, 660, 660, 1031, 111, 111, 289, 111, 111, 111, 730, 1035, 116, 1031, 120, 815, 815, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 289, 167, 473, 140, 143, 553, 157, 906, 178, 178, 166, 166, 257, 172, 172, 289, 172, 172, 172, 172, 479, 165, 182, 182, 182, 182, 182, 182, 182, 717, 183, 183, 183, 183, 183, 183, 183, 183, 183, 183, 1031, 187, 187, 188, 189, 189, 293, 190, 190, 190, 190, 190, 462, 289, 34, 197, 769, 206, 2

In [64]:
correct_num = 0

for i in range(0, len(predict_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

508
0.5866050808314087
